# Кластеризация текстов

На данном этапе работы над проектом MTScup я поставила задачу объединения имеющихся url в более крупные группы с целью минимизации потери информации (как это происходит в baseline). Для этого мне нужно было получить какую-то информацию о сайтах, имеющихся в нашей базе.

В рамках работы нашей команды  была принято решение использовать метод парсинга title с имеющихся страниц. (этот этап выполнил мой коллега). После сбора данных я собрала все тайтлы в одном датасете. и предобратала их на предмет пропусков и неинформативных заголовков страниц.

"Информативные"  url - я векторизую и кластеризую методом к-means на 50-150-300-500 кластеров. При дальнейшей работе я протестирую какой вариант являтся оптимальным для глобальной задачи

Те url, по которым не удалось получить информации и с невысокой посещаемостью пользователями,  на следующем этапе я буду обрабатывать с помощью получения разреженной матрицы и снижения ее размерности.

Url, без тайтлов, но с высокой посещаемостью будут обрабатываться как отедельные кластеры. 

Перед векторизацией я определяю язык текста и определяю язык-доминант (rus) и перевожу остальные тексты на него.


In [1]:
import pandas as pd

from deep_translator import GoogleTranslator
from tqdm import notebook 
import langid
import re
from pymystem3 import Mystem
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
import nltk 
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/makkate/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
import fasttext
fasttext.FastText.eprint = lambda x: None


In [3]:
df = pd.read_csv('titles_sib')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199682 entries, 0 to 199681
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  199682 non-null  int64  
 1   url         199682 non-null  object 
 2   us_count    199682 non-null  int64  
 3   title       199682 non-null  object 
 4   is_parced   199682 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 7.6+ MB


## Работа с titles (well parced)

Так как не со всех адресов удалось собрать корректно titles(где-то недоступен ресурс, где-то title был неинофрмативен или был вовсе пустой - данную предобработку можно найти в файле:) дальнейшая работа с titles мне видится следующим образом: где они собраны корректно - определеяем я

### Предобработка корректных titles


Обработаем строки с имеющимися title - определим язык и переведем все на русский 


In [5]:
model = fasttext.load_model('lid.176.ftz')
def detect_lang_err(x):
    try:
        lang = str(model.predict(x, k=1)[0][0])
        lang = lang.replace("__label__","")
        
        proba = model.predict(x, k=1)[1]
        return lang,proba
    except:
        return "error",'error'

In [6]:
%%time
language_detection=\
df.loc[df['is_parced']==-1]['title'].apply(lambda x: detect_lang_err(x))

CPU times: user 8.11 s, sys: 108 ms, total: 8.22 s
Wall time: 8.66 s


In [7]:
language_detection_1=pd.DataFrame(language_detection, index = df.loc[df['is_parced']==-1]['title'].index)

In [8]:
language_detection_1['lang'] = language_detection_1['title'].apply(lambda x: x[0])
language_detection_1['proba'] = language_detection_1['title'].apply(lambda x: x[1])

In [9]:
language_detection_1.loc[language_detection_1['title']==("error",'error'),"lang"]="Nan"
language_detection_1.loc[language_detection_1['title']==("error",'error'),"proba"]="Nan"

In [10]:
language_detection_1['proba']=language_detection_1['proba'].astype('float')
language_detection_1=language_detection_1.drop(['title'],axis=1)
#language_detection_1

In [11]:

df=df.join(language_detection_1)

In [12]:
df

,Unnamed: 0,url,us_count,title,is_parced,lang,proba
0,0,googleads.g.doubleclick.net,22013466,googleads.g.doubleclick.net,-2.0,NaN,NaN
1,1,yandex.ru,19007657,yandex.ru,-2.0,NaN,NaN
2,2,i.ytimg.com,16901446,i.ytimg.com,-2.0,NaN,NaN
3,3,vk.com,16695251,vk.com,-2.0,NaN,NaN
4,4,avatars.mds.yandex.net,16212095,avatars.mds.yandex.net,-2.0,NaN,NaN
...,...,...,...,...,...,...,...
199677,199678,money.poprostomu.com,1,money.poprostomu.com,-2.0,NaN,NaN
199678,199679,money.irktorgnews.ru,1,"Новости про деньги: личные, бизнеса и чужие - ...",-1.0,ru,0.928988
199679,199680,monety-10-50.blogspot.com,1,"Монеты России, СССР и Империи",-1.0,ru,0.984342
199680,199681,monetainfo.ru,1,МонетаИнфо - Всё для нумизматов,-1.0,ru,0.985585


In [13]:
def translate_lang_err(x):
    try:
        result = GoogleTranslator(source='auto', target='ru').translate(x)
    except:
        result ='error'
    return result

In [14]:
df['title_ru']=""

In [15]:
#Примем уровень вероятности при определении языка, при котором мы оставляем текст неизменным 80% и более
#сохраним для этих строк исходный title
df.loc[(df['lang']=="ru")&(df['proba']>=0.8),'title_ru'] = df.loc[(df['lang']=="ru")&(df['proba']>=0.8),'title']

In [16]:
#df.loc[((df['lang']=="ru")&(df['proba']<0.8))|((df['lang']!="ru")&(df['is_parced']==-1))]

In [17]:
#Определим длину всего массива, подлежащего переводу для дальнейшего разделения на батчи
len_trans=len(df.loc[((df['lang']=="ru")&(df['proba']<0.8))|((df['lang']!="ru")&(df['is_parced']==-1))])

In [18]:
len_trans

30586

In [ ]:
batch_size = 100
trans=[]
for i in notebook.tqdm(range(len_trans // batch_size)):
    
    blank=df.loc[((df['lang']=="ru")&(df['proba']<0.8))|((df['lang']!="ru")&(df['is_parced']==-1)),'title'].\
    iloc[batch_size*i:batch_size*(i+1)].\
    apply(lambda x: translate_lang_err(x))
    trans.extend(blank)
    #так как выполнение может прерваться по внешним причинам добавим промежуточное сохранение во внешний файл
    x_p=pd.DataFrame(trans)
    x_p.to_csv('check_trans')



  0%|          | 0/305 [00:00<?, ?it/s]

In [ ]:
blank=df.loc[((df['lang']=="ru")&(df['proba']<0.8))|((df['lang']!="ru")&(df['is_parced']==-1)),'title'].\
    iloc[len(trans):len_trans].\
    apply(lambda x: translate_lang_err(x))
trans.extend(blank)

In [ ]:
trans = pd.DataFrame(trans,index = df.loc[((df['lang']=="ru")&(df['proba']<0.8))|((df['lang']!="ru")&(df['is_parced']==-1)),'title'].index)
trans.columns=['title_ru_tr']

In [ ]:
trans

In [ ]:
df=df.join(trans)

In [ ]:
df.loc[(df['title_ru']=="")&(~(df['title_ru_tr'].isna())),'title_ru']=\
df.loc[(df['title_ru']=="")&(~(df['title_ru_tr'].isna())),'title_ru_tr']



In [ ]:
df.info()

In [ ]:
df.to_csv('titles_sib.csv')

In [ ]:
df.loc[(df['is_parced']==-1),['url','title_ru']]

### Кластеризация сайтов с валидными titles

С целью снижения признакового пространства и увеличения информативности малопосещаемых сайтов я хочу по имеющимся данных о тематике сайтов (информация из html title) "укрупнить" параметры посреством объединения их в кластеры. Близость будет изменяться между векторами, которые мы получим после векторизации строк методом Count Vectorizer.

In [ ]:
#создадим переменную со валидными тайтлами
df_cluster =df.loc[(df['is_parced']==-1),['url','title_ru']]

In [ ]:
#задаю функцию очистки текста от неинформативынх знаков
def clear_text(text):
    t=re.sub(r"[^А-Яа-яЁёA-Za-z']", ' ', text)
    t=" ".join(t.split())
    return t.lower()

In [ ]:
%%time
df_cluster['clear_text']=df_cluster['title_ru'].apply(lambda x: clear_text(x))
#очищаем тексты 

In [ ]:
#задаю функцию лемматизации текста
m = Mystem()
def lemmatize(text):
    
    
    
    lemm_list =m.lemmatize(text)
    lemm_text = " ".join(lemm_list)
        
    return clear_text(lemm_text)

In [ ]:
#пометим в исходном файле что данные сайты не информативны
df.loc[df_cluster.loc[df_cluster['clear_text']==""].index,'is_parced']=-2

In [ ]:
#отфильтруем неинформативные строки
df_cluster=df_cluster.loc[df_cluster['clear_text']!=""]

In [ ]:

%%time
df_cluster['lemm_text']=df_cluster['clear_text'].apply(lambda x: lemmatize(x))
#лемматизируем тексты

In [ ]:
#df_cluster.to_csv("df_cluster.csv")

In [ ]:
#получим мешок слов + приведем все к unicode
%%time
corpus = df_cluster['lemm_text'].values.astype('U')

In [ ]:
#выгрузим список стоп-слов из библиотеки nltk
stop_words = set(stopwords.words('russian'))

Поскольку имющиеся у нс тексты имеют определенную тематику, то помимо стандартных стоп-слов я считаю необходимо дабавить и те, которые неинформативны для нашего кейса (а-ля "страница" "ру" и пр) для этого получим список всех слов в корпусе и проранжируем ихх по встречаемовти - из топ 100 в ручном режиме отбираю слова-мусор, а так же отсекаю слова, встречающиеся менее 5 раз в массиве.

In [ ]:
explore_stop_words=[]
for i in corpus:
    for j in i.split():
        explore_stop_words.append(j)
        
explore_stop_words=pd.DataFrame(explore_stop_words)
explore_stop_words=explore_stop_words[0].value_counts()
explore_stop_words=pd.DataFrame(explore_stop_words)
explore_stop_words=explore_stop_words.reset_index()
explore_stop_words.columns = ['word','count']

In [ ]:
explore_stop_words=\
explore_stop_words.loc[~explore_stop_words['word'].isin(stop_words)]

In [ ]:
new_stop_words=['главный','ru','сайт','страница','г','ру','весь']
new_stop_words.extend(explore_stop_words.loc[explore_stop_words['count']<=5,'word'])

In [ ]:
for i in new_stop_words:
    stop_words.add(i)

Полученный список стоп-слов использую при векторизации. Использую count-vectorizer, вместо tf-idf так как дальнешая задача стостит не в семантическом анализе, а в поиске близостей текстов.

In [ ]:
count_vect = CountVectorizer(stop_words=list(stop_words))
count_vect.fit(corpus)
corp_vect=count_vect.transform(corpus)

***кластеризация текстов***

Поскольку заранее мне неизвестно исходное кол-во групп сайтов в которые мы хоти их объединить я буду использовать метод k-means и спрогнзирую три раза кластеризацию (на 50 тематик, на 150,на 300 и на 500) и уже при дальнейшей работе с предсказанием пола и возрата ползователей выберу вариант, обеспечивающий нас лучшими параметрами + оптимальный по времени работы. 

In [ ]:
from sklearn.cluster import MiniBatchKMeans

In [ ]:
mbk  = MiniBatchKMeans( n_clusters=50)
mbk.fit(corp_vect)
predict_k50=mbk.predict(corp_vect)
predict_k50=pd.DataFrame(predict_k50,index=df_cluster.index)
predict_k50.columns=['kmeans_50']
df_cluster=df_cluster.join(predict_k50)
#df_cluster.to_csv("df_cluster.csv")

In [ ]:
mbk  = MiniBatchKMeans( n_clusters=150)
mbk.fit(corp_vect)
predict_k150=mbk.predict(corp_vect)
predict_k150=pd.DataFrame(predict_k150,index=df_cluster.index)
predict_k150.columns=['kmeans_150']
df_cluster=df_cluster.join(predict_k150)
#df_cluster.to_csv("df_cluster.csv")

In [ ]:
mbk  = MiniBatchKMeans( n_clusters=300)
mbk.fit(corp_vect)
predict_k300=mbk.predict(corp_vect)
predict_k300=pd.DataFrame(predict_k300,index=df_cluster.index)
predict_k300.columns=['kmeans_300']
df_cluster=df_cluster.join(predict_k300)
#df_cluster.to_csv("df_cluster.csv")

In [ ]:
mbk  = MiniBatchKMeans( n_clusters=500)
mbk.fit(corp_vect)
predict_k500=mbk.predict(corp_vect)
predict_k500=pd.DataFrame(predict_k500,index=df_cluster.index)
predict_k500.columns=['kmeans_500']
df_cluster=df_cluster.join(predict_k500)
#df_cluster.to_csv("df_cluster.csv")

In [ ]:
df_cluster=df_cluster[['lemm_text','kmeans_50','kmeans_150','kmeans_300','kmeans_500']]

In [ ]:
df.loc[(df['is_parced']==-1),['url','title_ru']]

In [ ]:
df=df.join(df_cluster)

## Предобработка адресов, с отсутствующими title

Прежде чем кластеризовать по адресам сначала разделлим адреса, которые пользуются популярностью - их я оставлю в исходном виде(как самостоятельные), так как они сами по себе имеют большой вес для дальнейшего предсказания таргетов (пол и возраст),а вот те сайты, которые имеют мало посещений  - есть смысл объединить со схожими.

In [ ]:
df.loc[df['is_parced']==-2,'us_count'].describe()

In [ ]:
df.loc[df['is_parced']==-2,'us_count'].plot(kind='box');

In [ ]:
trash = df['us_count'].sort_values(ascending=False)[500]

In [ ]:
trash

Разделим сайты на 2 группы: 1 - сайты с достаточно высокой посещаемостью 2 - c низкой посещаемостью
примем за условный разделитель: топ 500 сайтов 

In [ ]:
df.loc[(df['is_parced']==-2)&((df['us_count']>trash)),'us_count'].hist();

In [ ]:
df.loc[(df['is_parced']==-2)&((df['us_count']<=trash)),"us_count"].hist();

In [ ]:
df['unparced_url_type']=""

In [ ]:
df.loc[(df['is_parced']==-2)&((df['us_count']<=trash)),'unparced_url_type']="low_usage"
df.loc[(df['is_parced']==-2)&((df['us_count']>trash)),'unparced_url_type']="high_usage"

In [ ]:
df['unparced_url_type']=df['unparced_url_type'].fillna('parces_url_check_cluster')

In [ ]:
df.info()

## Подотовка таблицы для экспорта

Поскольку топ-500 сайтов мы будем считать за отдельные неизменыемые кластеры, то с целью дальнейшей оптимизации хочу их добавить в уже имеющиеся кластеры "k-means..." под споими именами. там образом в в каждом варианте кластеризации мы увеличим кол-во кластеров на 500шт.

In [ ]:
df_fin=df[['url','unparced_url_type','kmeans_50','kmeans_150','kmeans_300','kmeans_500']]
df_fin=df_fin.set_index('url')

In [ ]:
df_fin.loc[df_fin['unparced_url_type']=="high_usage",'kmeans_50']=\
df_fin.loc[df_fin['unparced_url_type']=="high_usage",'kmeans_50'].index

In [ ]:
df_fin.loc[df_fin['unparced_url_type']=="high_usage",'kmeans_150']=\
df_fin.loc[df_fin['unparced_url_type']=="high_usage",'kmeans_150'].index

In [ ]:
df_fin.loc[df_fin['unparced_url_type']=="high_usage",'kmeans_300']=\
df_fin.loc[df_fin['unparced_url_type']=="high_usage",'kmeans_300'].index

In [ ]:
df_fin.loc[df_fin['unparced_url_type']=="high_usage",'kmeans_500']=\
df_fin.loc[df_fin['unparced_url_type']=="high_usage",'kmeans_500'].index

In [ ]:
df_fin.columns=['is_low_usage', 'kmeans_50', 'kmeans_150', 'kmeans_300',
       'kmeans_500']

In [ ]:
df_fin.loc[df_fin['is_low_usage']=="high_usage",'is_low_usage']=False
df_fin.loc[df_fin['is_low_usage']=="low_usage",'is_low_usage']=True
df_fin.loc[df_fin['is_low_usage']=="",'is_low_usage']=False

In [ ]:
df_fin.info()

In [ ]:
df_fin.to_csv('url_clusters.csv')

# Выводы

Все задачи данного этапа выполнены:
1. обработаны заголовки страниц и приведены в векторы
2. произведена кластеризация в нескольких вариантах
3. крупные сайты были выделены в отдельные кластеры
4. мелкие сайты без валидных оглавлений были помечены для удобства дальнейшей рыботы

## Следующий этап работы см. файл : mts_ml_cup_sibrikova_main_body.ipynb